# Sofxmax分类器

## 背景

通过使用softmax分类器构建神经网络。

## 准备工作

1.创建一个SBT项目，并引入相关依赖（参照[Getting Started](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started) 或者将下面的依赖引入build.sbt, 注意DeepLearning.scala暂不支持scala2.12.X )
```
libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableany" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablenothing" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableseq" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiabledouble" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablefloat" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablehlist" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablecoproduct" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableindarray" % "latest.release"

addCompilerPlugin("com.thoughtworks.implicit-dependent-type" %% "implicit-dependent-type" % "latest.release")

addCompilerPlugin("org.scalamacros" % "paradise" % "2.1.0" cross CrossVersion.full)

fork := true
```
2.[下载CIFAR-10 binary version (suitable for C programs)](https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz)，文件大小162 MB，md5sum：c32a1d4ab5d03f1284b67883e8d87530

3.将下载好的文件解压到src/main/resources目录。

4.Scala类ReadCIFAR10ToNDArray用于从上面的文件中读取图片及其标签数据并做归一化处理（[更多信息](https://www.cs.toronto.edu/~kriz/cifar.html)）：

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:1.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC5`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.{DifferentiableHList,DifferentiableINDArray,Layer}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Lift.Layers.Identity
import com.thoughtworks.deeplearning.Lift._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq

pprintConfig() = pprintConfig().copy(height = 5)//减少输出的行数，避免页面输出太长

import $file.ReadCIFAR10ToNDArray,ReadCIFAR10ToNDArray._

import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.DifferentiableHList._

import com.thoughtworks.deeplearning

5.[softmax](https://en.wikipedia.org/wiki/Softmax_function): softmax是[logistic](https://en.wikipedia.org/wiki/Logistic_function) 对多分类的一般化归纳。公式：![](https://www.zhihu.com/equation?tex=f_j%28z%29%3D%5Cfrac%7Be%5E%7Bz_j%7D%7D%7B%5Csum_ke%5E%7Bz_k%7D%7D)

   [交叉熵损失（cross-entropy loss）](https://en.wikipedia.org/wiki/Cross-entropy):p表示真实标记的分布，q则为训练后的模型的预测标记分布，交叉熵损失函数可以衡量p与q的相似性。公式：![](https://zhihu.com/equation?tex=%5Cdisplaystyle+H%28p%2Cq%29%3D-%5Csum_xp%28x%29+logq%28x%29)
   
6.如果你使用IntelliJ或者eclipse等其它IDE，智能提示可能会失效，代码有部分可能会爆红，这是IDE的问题，代码本身并无问题。
   

## 构建神经网络

1.新建一个Scala类SoftmaxLinearClassifier

2.从CIFAR10 database中读取训练数据和测试数据的图片和标签信息

In [2]:
  //CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
  val NumberOfClasses: Int = 10

  //加载train数据,我们读取1000条数据作为训练数据
  val trainNDArray =
    ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/data_batch_1.bin", 1000)

  //加载测试数据，我们读取100条作为测试数据
  val testNDArray =
    ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


NumberOfClasses: Int = 10
trainNDArray: INDArray :: INDArray :: HNil = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0.66, 0.62, 0.62, 0.62, 0.59, 0.58, 0.06, 0.00, 0.07, 0.20, 0.34, 0.47, 0.50, 0.50, 0.49, 0.45, 0.41, 0.39, 0.41, 0.44, 0.43, 0.44, 0.46, 0.43, 0.41, 0.49, 0.50, 0.48, 0.51, 0.48, 0.47, 0.51, 0.52, 0.52, 0.52, 0.48, 0.46, 0.48, 0.10, 0.06, 0.19, 0.32, 0.43, 0.50, 0.51, 0.47, 0.44, 0.44, 0.44, 0.41, 0.41, 0.50, 0.48, 0.51, 0.50, 0.48, 0.45, 0.47, 0.51, 0.51, 0.54, 0.50, 0.49, 0.50, 0.51, 0.55, 0.51, 0.46, 0.47, 0.43, 0.13, 0.15, 0.34, 0.41, 0.45, 0.46, 0.45, 0.41, 0.42, 0.47, 0.49, 0.43, 0.44, 0.57, 0.52, 0.50, 0.46, 0.46, 0.50, 0.48, 0.52, 0.54, 0.53, 0.51, 0.48, 0.51, 0.52, 0.53, 0.51, 0.49, 0.47, 0.37, 0.20, 0.23, 0.40, 0.50, 0.48, 0.47, 0.47, 0.45, 0.42, 0.49, 0.50, 0.41, 0.42, 0.48, 0.47, 0.42, 0.38, 0.43, 0.46, 0.47, 0.52, 0.55, 0.51, 0.55, 0.53, 0.50, 0.4

3.编写处理标签数据的工具方法，将N行一列的NDArray转换为N行NumberOfClasses列的NDArray，每行对应的正确分类的值为1，其它列的值为0。这样做是为了向cross-entropy loss公式靠拢

In [3]:
  /**
    * 处理标签数据：将N行一列的NDArray转换为N行NumberOfClasses列的NDArray，每行对应的正确分类的值为1，其它列的值为0
    *
    * @param ndArray 标签数据
    * @return N行NumberOfClasses列的NDArray
    */
  def makeVectorized(ndArray: INDArray): INDArray = {
    val shape = ndArray.shape()

    val p = Nd4j.zeros(shape(0), NumberOfClasses)
    for (i <- 0 until shape(0)) {
      val double = ndArray.getDouble(i, 0)
      val column = double.toInt
      p.put(i, column, 1)
    }
    p
  }

defined function makeVectorized

4.分离和处理图像和标签数据

In [4]:
  val train_data = trainNDArray.head
  val test_data = testNDArray.head

  val train_expect_result = trainNDArray.tail.head
  val test_expect_result = testNDArray.tail.head
  
  val p = makeVectorized(train_expect_result)
  val test_p = makeVectorized(test_expect_result)

train_data: INDArray = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0.66, 0.62, 0.62, 0.62, 0.59, 0.58, 0.06, 0.00, 0.07, 0.20, 0.34, 0.47, 0.50, 0.50, 0.49, 0.45, 0.41, 0.39, 0.41, 0.44, 0.43, 0.44, 0.46, 0.43, 0.41, 0.49, 0.50, 0.48, 0.51, 0.48, 0.47, 0.51, 0.52, 0.52, 0.52, 0.48, 0.46, 0.48, 0.10, 0.06, 0.19, 0.32, 0.43, 0.50, 0.51, 0.47, 0.44, 0.44, 0.44, 0.41, 0.41, 0.50, 0.48, 0.51, 0.50, 0.48, 0.45, 0.47, 0.51, 0.51, 0.54, 0.50, 0.49, 0.50, 0.51, 0.55, 0.51, 0.46, 0.47, 0.43, 0.13, 0.15, 0.34, 0.41, 0.45, 0.46, 0.45, 0.41, 0.42, 0.47, 0.49, 0.43, 0.44, 0.57, 0.52, 0.50, 0.46, 0.46, 0.50, 0.48, 0.52, 0.54, 0.53, 0.51, 0.48, 0.51, 0.52, 0.53, 0.51, 0.49, 0.47, 0.37, 0.20, 0.23, 0.40, 0.50, 0.48, 0.47, 0.47, 0.45, 0.42, 0.49, 0.50, 0.41, 0.42, 0.48, 0.47, 0.42, 0.38, 0.43, 0.46, 0.47, 0.52, 0.55, 0.51, 0.55, 0.53, 0.50, 0.47, 0.46, 0.40, 0.34, 0.29, 0.26...
test_data: IN

5.编写softmax函数,和准备一节中的softmax公式对应

In [5]:
  def softmax(implicit scores: From[INDArray] ##T): To[INDArray] ##T = {
    val expScores = exp(scores)
    expScores / expScores.sum(1)
  }

defined function softmax

6.设置学习率，学习率是Weight变化的快慢的直观描述，学习率设置的过小会导致loss下降的很慢，需要更长时间来训练，学习率设置的过大虽然刚开始下降很快但是会导致在接近最低点的时候在附近徘徊loss下降会非常慢。

In [6]:
  implicit def optimizer: Optimizer = new LearningRate {
    def currentLearningRate() = 0.00001
  }

defined function optimizer

7.跟定义一个方法一样定义一个神经网络并初始化Weight，Weight应该是一个N*NumberOfClasses的INDArray,每个图片对应每个分类都有一个评分。[什么是Weight](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started#231--weight-intialization)

In [7]:
  def createMyNeuralNetwork(implicit input: From[INDArray] ##T): To[INDArray] ##T = {
    val initialValueOfWeight = Nd4j.randn(3072, NumberOfClasses) * 0.001
    val weight: To[INDArray] ##T = initialValueOfWeight.toWeight
    val result: To[INDArray] ##T = input dot weight
    softmax.compose(result) //对结果调用softmax方法，压缩结果值在0到1之间方便处理
  }
  val myNeuralNetwork: FromTo[INDArray, INDArray] ##T = createMyNeuralNetwork

defined function createMyNeuralNetwork
myNeuralNetwork: (FromTo[INDArray, INDArray]{type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray;type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray})#T = Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),Dot(Identity(),Weight([[-0.00, 0.00, 0.00, 0.00, 0.00, 0.00, -0.00, 0.00, -0.00, -0.00],
 [-0.00, 0.00, 0.00, 0.00, 0.00, -0.00, 0.00, -0.00, -0.00, -0.00],
 [0.00, 0.00, 0.00, -0.00, -0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, -0.00, -0.00, 0.00, -0.00, -0.00, 0.00, -0.00, 0.00, -0.00],
 [0.00, -0.00, -0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, -0.00],
 [-0.00, -0.00, -0.00, -0.00, 0.00, -0.00, -0.00, -0.00, -0.00, -0.00],
 [0.00, 0.00, -0.00, -0.00, 0.00, 0.00, 0.00, 0.00, 0.00, -0.00],
 [-0.00, -0.00, -0.00, 0.00, -0.00, 0.00, 0.00, 0.00, 0.00, -0.00],
 [0.00, -0.00, -0.00, 0.00, 0.00, 0.

8.编写损失函数Loss Function，将此次判断的结果和真实结果进行计算得出cross-entropy loss并返回

In [8]:
  def lossFunction(implicit pair: From[INDArray :: INDArray :: HNil] ##T): To[Double] ##T = {
    val input = pair.head
    val expectedOutput = pair.tail.head
    val probabilities = myNeuralNetwork.compose(input)

    -(expectedOutput * log(probabilities)).sum //此处和准备一节中的交叉熵损失对应
  }

defined function lossFunction

9.训练神经网络并观察每次训练loss的变化，loss的变化趋势应该是越来越低的

In [9]:
  val lossSeq = for (_ <- 0 until 2000) yield {
    lossFunction.train(train_data :: p :: HNil)
  }

  plotly.JupyterScala.init()
  val plot = Seq(
    Scatter(
      0 until 2000 by 1,
      lossSeq
    )
  )

  plot.plot(
    title = "loss on time"
  )

lossSeq: Seq[Double] = List(
  2304.05615234375,
  2284.93359375,
  2275.099365234375,
  2266.16552734375,
  2257.55029296875,
  2249.195068359375,
  2241.08251953125,
  2233.203125,
  2225.54736328125,
  2218.10400390625,
  2210.8662109375,
...
plot: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
          0.0,
          1.0,
          2.0,
          3.0,
          4.0,
          5.0,
          6.0,
...
res8_4: String = "plot-2000235454"

10.使用训练后的神经网络判断测试数据的标签

In [10]:
  val result = myNeuralNetwork.predict(test_data)
  println(s"result: $result") //输出判断结果

result: [[0.03, 0.05, 0.17, 0.13, 0.01, 0.13, 0.42, 0.00, 0.04, 0.00],
 [0.03, 0.17, 0.00, 0.05, 0.00, 0.01, 0.00, 0.00, 0.18, 0.55],
 [0.08, 0.09, 0.01, 0.03, 0.01, 0.00, 0.00, 0.01, 0.71, 0.05],
 [0.50, 0.04, 0.07, 0.04, 0.01, 0.02, 0.00, 0.03, 0.21, 0.08],
 [0.02, 0.03, 0.09, 0.05, 0.36, 0.10, 0.25, 0.08, 0.01, 0.00],
 [0.01, 0.07, 0.02, 0.62, 0.02, 0.04, 0.14, 0.05, 0.00, 0.03],
 [0.00, 0.00, 0.00, 0.69, 0.02, 0.05, 0.23, 0.00, 0.00, 0.00],
 [0.04, 0.01, 0.24, 0.04, 0.26, 0.10, 0.17, 0.11, 0.01, 0.01],
 [0.05, 0.17, 0.22, 0.09, 0.17, 0.16, 0.09, 0.03, 0.01, 0.01],
 [0.09, 0.44, 0.02, 0.03, 0.00, 0.06, 0.01, 0.01, 0.11, 0.24],
 [0.33, 0.09, 0.04, 0.09, 0.02, 0.07, 0.04, 0.02, 0.28, 0.03],
 [0.02, 0.49, 0.02, 0.05, 0.01, 0.01, 0.01, 0.03, 0.04, 0.33],
 [0.02, 0.32, 0.02, 0.24, 0.12, 0.07, 0.15, 0.07, 0.01, 0.01],
 [0.04, 0.21, 0.03, 0.07, 0.05, 0.01, 0.15, 0.01, 0.29, 0.15],
 [0.24, 0.18, 0.05, 0.17, 0.02, 0.04, 0.03, 0.13, 0.07, 0.06],
 [0.19, 0.03, 0.19, 0.02, 0.09, 0.06, 0.03, 0.0

result: FromTo.<refinement>.this.type.OutputData = [[0.03, 0.05, 0.17, 0.13, 0.01, 0.13, 0.42, 0.00, 0.04, 0.00],
 [0.03, 0.17, 0.00, 0.05, 0.00, 0.01, 0.00, 0.00, 0.18, 0.55],
 [0.08, 0.09, 0.01, 0.03, 0.01, 0.00, 0.00, 0.01, 0.71, 0.05],
 [0.50, 0.04, 0.07, 0.04, 0.01, 0.02, 0.00, 0.03, 0.21, 0.08],
 [0.02, 0.03, 0.09, 0.05, 0.36, 0.10, 0.25, 0.08, 0.01, 0.00],
 [0.01, 0.07, 0.02, 0.62, 0.02, 0.04, 0.14, 0.05, 0.00, 0.03],
 [0.00, 0.00, 0.00, 0.69, 0.02, 0.05, 0.23, 0.00, 0.00, 0.00],
 [0.04, 0.01, 0.24, 0.04, 0.26, 0.10, 0.17, 0.11, 0.01, 0.01],
 [0.05, 0.17, 0.22, 0.09, 0.17, 0.16, 0.09, 0.03, 0.01, 0.01],
 [0.09, 0.44, 0.02, 0.03, 0.00, 0.06, 0.01, 0.01, 0.11, 0.24],
 [0.33, 0.09, 0.04, 0.09, 0.02, 0.07, 0.04, 0.02, 0.28, 0.03],
 [0.02, 0.49, 0.02, 0.05, 0.01, 0.01, 0.01, 0.03, 0.04, 0.33],
...

11.编写工具方法，从一行INDArray中获得值最大的元素所在的列，目的是获得神经网络判断的结果，方便和原始标签比较以得出正确率。

In [11]:
  /**
    * 从一行INDArray中获得值最大的元素所在的列
    * @param iNDArray
    * @return
    */
  def findMaxItemIndex(iNDArray: INDArray): Int = {
    val shape = iNDArray.shape()
    val col = shape(1)
    var maxValue = 0.0
    var maxIndex = 0
    for (index <- 0 until col) {
      val itemValue = iNDArray.getDouble(0, index)
      if (itemValue > maxValue) {
        maxValue = itemValue
        maxIndex = index
      }
    }
    maxIndex
  }

defined function findMaxItemIndex

12.判断神经网络对测试数据分类判断的正确率，正确率应该在32%左右。

In [12]:
  var right = 0

  val shape = result.shape()
  for (row <- 0 until shape(0)) {
    val rowItem = result.getRow(row)
    val index = findMaxItemIndex(rowItem)
    if (index == test_expect_result.getDouble(row, 0)) {
      right += 1
    }
  }
  println(s"the result is $right %")

the result is 32 %


right: Int = 32
shape: Array[Int] = Array(100, 10)


13.[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/SoftmaxLinearClassifier.scala)